#JPX STOCK Prediction Explaratory Data Analysis


How to get setup up and running: 
1. run 'jupyter-lab' in the command line
2. open another shell and cd to working directory
3. git status to check if there are pending updates
4. git add . (learn to use other 'adds' and 'gitignore')
5. git commit -m "type a message here"
6. git push

##Importing libraries and packages

In [ ]:
# Import Dependencies
%matplotlib inline

# Start Python Imports
import math, time, random, datetime

# Data Manipulation
import numpy as np
import pandas as pd

# Visualization 
import matplotlib.pyplot as plt
import missingno
import seaborn as sns
plt.style.use('seaborn-whitegrid')

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize

# Machine learning
import catboost
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier, Pool, cv

# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')

##Loading the data

The data for this competition is too big to be updated on Github.
We will store the data in a separate directory in the parent folder 

I ended up copying the information into this folder then using .gitignore to avoid publishing data into github

In [ ]:
#Import train & test data & more
stock_list = pd.read_csv('data/stock_list.csv')
df_financials = pd.read_csv('data/train_files/financials.csv')
df_options = pd.read_csv('data/train_files/options.csv')
df_secondary_stock_prices = pd.read_csv('data/train_files/secondary_stock_prices.csv')
df_stock_prices = pd.read_csv('data/train_files/stock_prices.csv')
df_trades = pd.read_csv('data/train_files/trades.csv')

##Adding head and missing table values for the stock_list

In [ ]:
stock_list.head(5)

##Outputting head(15) of the training data -- 
Still in phase of understanding what everything means

In [ ]:
# Plot graphic of missing values
missingno.matrix(stock_list, figsize = (40,10))

In [ ]:
# Another way of checking how many missing values are in a dataset
stock_list.isnull().sum()

In [ ]:
#df_stock_prices.head(5)
df_stock_prices.describe()

In [ ]:
# Plot graphic of missing values
missingno.matrix(df_stock_prices, figsize = (40,10))

In [ ]:
# Another way of checking how many missing values are in a dataset
df_stock_prices.isnull().sum()

##Notice
There are a lot of missing values in the ExpectedDividend column, therefore we cannot make an accurate judgment with it as a variable during training
##To-do
Figure out why there are 7608 stocks without OPEN/HIGH/LOW columns (assuming they are common)
Write a program that groups these stocks together and try to find a pattern

In [ ]:
df_trades.head(5)

In [ ]:
df_secondary_stock_prices.head(5)

In [ ]:
df_options.head(5)

##Understanding the data
Read from the data_specification sub-folder. Trying to summarize the definition of each specific column for understanding and future reference

###Options specifications
What is night session? 
https://www.jpx.co.jp/english/derivatives/rules/trading-hours/index.html

The Japanese market operates a night session for different options/futures to allow trade between overseas market (US, Europe) 

Whole & Day & Night / Low & High & Close & Open - all floats

###Stock_List specifications


##Understanding the scoring system

The first 200 stocks and the last 200 out of a list of 2000 will be bought and shorted, respectively.

The sum of the first 200 stocks will be saved into a variable called S_{up}. The ranking of the stock will determine its linear weight which goes from 2-to-1.
This means that the first stock has a weight of 2 and the 200th a weight of 1. This allows competitors with the highest return on stock that were ranked high to get a greater S_{up} value.

The same process is done for the last 200 stocks (shorted) and stored into a variable called S_{down}.
The final score of the competitors is then simply S_{up}-S{down}.

Practicing outputs for data. 

In [ ]:
code = 1333
price_data = df_stock_prices[df_stock_prices["SecuritiesCode"] == code]
print(price_data.count())

fig, ax = plt.subplots(figsize=(30, 15))

ax.plot(price_data['Date'],price_data["Open"], label=f"Open")
ax.plot(price_data['Date'],price_data["High"], label=f"High")
ax.plot(price_data['Date'],price_data["Low"], label=f"Low")
ax.plot(price_data['Date'],price_data["Close"], label=f"Close")
plt.xticks([0, 200, 400, 600, 800, 1000, 1200])
ax.set_ylabel("stock_price (yen)")
ax.set_xlabel("datetime (YYYY-MM-DD)")
ax.grid(True)
ax.legend()
